<a href="https://colab.research.google.com/github/rrizwan98/SSUET-chatbot/blob/main/Final_SSUET_chatbot_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import nessessory library
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/content/intents4.json').read()
intents = json.loads(data_file)

In [ ]:
for intent in intents['intents']:
    for pattern in intent['question']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['context']))

        # adding classes to our class list
        if intent['context'] not in classes:
            classes.append(intent['context'])
        print(pattern)
        

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

Hi
How are you
Is anyone there?
Hello
Good day
hey
hey there
Assalam-o-alaikum
Bye
See you later
Goodbye
Nice chatting to you, bye
Till next time
Thanks
Thank you
That's helpful
Awesome, thanks
Thanks for helping me
your name
chatbot
what is your name
purpose of this chatbot?
chatbot name
How you could help me?
What you can do?
What help you provide?
How you can be helpful?
What support is offered
Introduction to computing teacher name?
Introduction to computing credit hours?
Introduction to computing teacher room?
Sir Haris Mehboob room no

Introduction to computing course code
introduction to computing class room No?
introduction to computing lab teacher room no
Sir Haris Mehboob room no
introduction to computing lab teacher name?
itc teacher name?
introduction to computing Lab Room No?
introduction to computing time table
time table of itc
itc time table
itc class timing
introduction to computing lab time table
lab time table of itc
itc lab time table
itc lab timing
what is programm

In [ ]:
print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


784 documents
555 classes ['AI Lab room no', 'AI Lab teacher name', 'AI Lab teacherroom no', 'AI class room', 'AI course code', 'AI credit hours', 'AI teacher name', 'AI teacher room', 'AP class room ', 'AP course code ', 'AP credit hours ', 'AP teacher name', 'AP teacher room  ', 'ATFL class room', 'ATFL course code', 'ATFL credit hours', 'ATFL teacher name', 'ATFL teacher room', 'Applied Physics AP class timing time table', 'Applied Physics Lab room no ap lab room no', 'Applied Physics Lab teacher name ap Lab teacher name', 'Applied Physics Lab teacher room ap Miss Noreen teacher room no', 'Applied Physics class room ', 'Applied Physics course code ', 'Applied Physics credit hours ', 'Applied Physics lab timing AP lab timing', 'Applied Physics teacher name', 'Applied Physics teacher room no ', 'Artificial Intelliigence Engineering lab timing ai class timing ', 'Artificial Intelliigence Lab room no', 'Artificial Intelliigence Lab teacher name', 'Artificial Intelliigence Lab teacher ro

In [ ]:
pickle.dump(words,open('final_words.pkl','wb'))
pickle.dump(classes,open('final_classes.pkl','wb'))

In [ ]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [ ]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [ ]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
#fitting model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=16, verbose=1)

Epoch 1/200
49/49 [==============================] - 3s 4ms/step - loss: 6.3311 - accuracy: 0.0000e+00
Epoch 2/200
49/49 [==============================] - 0s 4ms/step - loss: 6.2897 - accuracy: 0.0089
Epoch 3/200
49/49 [==============================] - 0s 4ms/step - loss: 6.2132 - accuracy: 0.0102
Epoch 4/200
49/49 [==============================] - 0s 4ms/step - loss: 6.0298 - accuracy: 0.0115
Epoch 5/200
49/49 [==============================] - 0s 4ms/step - loss: 5.8119 - accuracy: 0.0128
Epoch 6/200
49/49 [==============================] - 0s 4ms/step - loss: 5.6204 - accuracy: 0.0204
Epoch 7/200
49/49 [==============================] - 0s 4ms/step - loss: 5.4569 - accuracy: 0.0268
Epoch 8/200
49/49 [==============================] - 0s 4ms/step - loss: 5.3512 - accuracy: 0.0395
Epoch 9/200
49/49 [==============================] - 0s 4ms/step - loss: 5.2125 - accuracy: 0.0421
Epoch 10/200
49/49 [==============================] - 0s 4ms/step - loss: 5.1170 - accuracy: 0.0434
Epoch

In [ ]:
# saving the model
model.save('final_chatbot_model1.h5', hist)

print("model created")

model created


In [ ]:
from flask import Flask, jsonify
# from flask_cors import CORS, cross_origin
import random
import json
from tensorflow.keras.models import load_model
import numpy as np
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


intents = json.loads(open('/content/intents4.json', encoding="utf8").read())
words = pickle.load(open('/content/final_words.pkl', 'rb'))
classes = pickle.load(open('/content/final_classes.pkl', 'rb'))


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(
        word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence


def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return(np.array(bag))


def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['context'] == tag):
            result = random.choice(i['answer'])
            break
    return result


def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

def decrypt(msg):

    string = msg

    # converting back '+' character back into ' ' spaces
    # new_string is the normal message with spaces that was sent by the user
    new_string = string.replace("+", " ")

    return new_string

In [ ]:
!pip install -U flask-cors


In [ ]:
from flask import Flask, jsonify
from flask_cors import CORS, cross_origin
import random
import json
from tensorflow.keras.models import load_model
import numpy as np
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
#from keras.models import load_model
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

model = load_model('/content/final_chatbot_model1.h5')

intents = json.loads(open('/content/intents4.json', encoding="utf8").read())
words = pickle.load(open('/content/final_words.pkl', 'rb'))
classes = pickle.load(open('/content/final_classes.pkl', 'rb'))


def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(
        word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence


def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return(np.array(bag))


def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['context'] == tag):
            result = random.choice(i['answer'])
            break
    return result


def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res


''' Flask code '''


app = Flask(__name__)
CORS(app)


@app.route("/", methods=['GET', 'POST'])
def hello():
    return jsonify({"key": "home page value"})

# function to replace '+' character with ' ' spaces


def decrypt(msg):

    string = msg

    # converting back '+' character back into ' ' spaces
    # new_string is the normal message with spaces that was sent by the user
    new_string = string.replace("+", " ")

    return new_string

# here we will send a string from the client and the server will return another
# string with som modification
# creating a url dynamically


@app.route('/home/<name>')
def hello_name(name):

    # dec_msg is the real question asked by the user
    dec_msg = decrypt(name)

    # get the response from the ML model & dec_msg as the argument
    response = chatbot_response(dec_msg)

    # creating a json object
    json_obj = jsonify({"top": {"res": response}})

    return json_obj


if __name__ == '__main__':
    app.run(debug=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
